#### If running in Google Colab, use the below route

+ the below path should be changed to the exact path that you save this script


In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/PLearning/Precision Learning-PSU Only/precisionLearning/code')

#### Combine all the files and clean
+ given files have different keys, first join the files with common 'video id' key
+ second, join with object file with the common key of 'video_title'
+ third, clean the file to have no more 

In [0]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [0]:
def data_combine(data_dir,init_path,marker):
    import pandas as pd
    import os
    if init_path==None:

        df=pd.DataFrame(columns=['Video id'])
    else:
        df=pd.read_csv(init_path)
    for file in os.listdir(data_dir):
        file=os.path.join(data_dir,file)
        print(file)
        data=pd.read_csv(file)
        
        df=pd.merge(data,df,how='outer',on='Video id')
        print('It has shape {} after merging'.format(df.shape))
    df.to_csv('../data/all_features_with_{}.csv'.format(marker),index=False)
    return df

df_185=data_combine('../data/185_row',None,'185row')

def combine_object(df_185)
    df_185=df_185.drop(['Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0_x','Unnamed: 0_y'],axis=1)
    print(df_185.shape)#(390, 134)
    df_185_dd=df_185.drop_duplicates()
    print(df_185_dd.shape)#(266, 134)
    df_185_dd.to_csv('all_features_with_185row_dd1.csv',index=False)
    # df_185_dd=pd.read_csv('all_features_with_185row_dd1.csv')
    print(df_185_dd.shape) 
    df_185_dd['video_title']=df_185_dd['video_title'].fillna('Missing')

    df_185_dd['video_title']=df_185_dd['video_title'].str.replace("[#|.|,|\||/|:]","").str.replace('(\&#39;)|(\&quot);',"").str.replace("&amp;","_")

    df_185_dd['video_title']=df_185_dd['video_title'].str.replace('CCSS 7 SP 3 video 1 Mean Median Mode','CCSS 7 SP 3   video 1   Mean Median Mode')
    df_185_dd['video_title']=df_185_dd['video_title'].str.replace('CCSS 7 SP 5 video 1 Understanding Probability','CCSS 7 SP 5   video 1   Understanding Probability')
    df_185_dd['video_title']=df_185_dd['video_title'].str.replace('I don&39;t get Constant','I dont get Constant')
    L1=list(df_185_dd['video_title'].unique())
    L1.sort()
    L1
    label_df_narrow=pd.read_csv('../data/video_objects.csv')
    df_185_all=pd.merge(df_185_dd,label_df_narrow,how='outer',on='video_title')
    print(df_185_all.shape)#(269, 154)
    df_185_all.to_csv('../data/all_features.csv',index=False)

    return df_185_all

def clean_data(df_185_all)
    data0=pd.get_dummies(df_185_all[['IsFace']],columns=['IsFace'])
    new_name=pd.Series(data0.columns.to_list()).str.replace('IsFace_','')
    df_185_all0=df_185_all.drop('IsFace',axis=1)
    print(df_185_all0.shape)
    data0.columns=new_name
    df_185_all=pd.concat([df_185_all0,data0],axis=1)
    print(df_185_all.shape)
    data0.head()
    temp=df_185_all.drop(['code_desc','transcript'],axis=1)
    df_185_all=df_185_all.drop_duplicates(subset='Video id')
    print(df_185_all.shape)
    df_185_all.head()#(190, 156)
    df_185_all.to_csv('../data/all_features_dd.csv',index=False)
    df_185_all1=df_185_all.dropna(subset=['Label'])
    df_185_all1.to_csv('../data/all_features_noLabelNA.csv',index=False)
    # aligning lables
    conditions=[df_185_all1['Label']=='Yes',
                df_185_all1['Label'].isin(['Not Relevant','Not Revelant']),
                (df_185_all1['Label']=='Not Educational')]
    choice=['Yes','No','No']
    df_185_all1['label']=np.select(conditions, choice)
    df_185_all1.drop('Label',axis=1,inplace=True)
    df_185_all1.to_csv('../data/all_features_cleaned.csv',index=False)
    return df_185_all1


#### Running 4 classifiers to see the best performance
+ algorithms are used: KNNClassifer, Decision Tree, Random Forest, XGBoost


In [0]:
def evaluate(x_train,x_test,y_train,y_test):
    import pandas as pd
    import numpy as np
    import sklearn
    from sklearn.model_selection import train_test_split
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from xgboost import XGBClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    import scikitplot as skplt
    import matplotlib.pyplot as plt
    %matplotlib inline
    import warnings
    warnings.filterwarnings('ignore')
# name of models
    model_name_list=['KNN','Decision Tree', 'Random Forest','XGboost']

    #instantiate the models
    model1=KNeighborsClassifier()
    model2=DecisionTreeClassifier(min_samples_split=2,random_state=0)
    model3=RandomForestClassifier(min_samples_split=2,random_state=0)
    model4=XGBClassifier(
                 learning_rate =0.01,
                #  max_depth=4,
                 min_child_weight=1,
                 gamma=0,
                 subsample=0.8,
                 colsample_bytree=0.8,
                 objective='binary:logistic',
                 n_jobs=4,
                 scale_pos_weight=1,
                 reg_alpha=0,
                 reg_lambda=1,
                 seed=100)

    #dataframe for results
#     results=pd.DataFrame(columns=['y_proba'],index=model_name_list)
    
    #train and predict with each model
    for i, model in enumerate([model1,model2, model3,model4]):
        model.fit(x_train, y_train)
        y_proba=model.predict_proba(x_test)
        #metrics

        y_pred=model.predict(x_test)
        #insert results into the dataframe
        model_name=model_name_list[i]
        print('----'+ model_name+'-----')
        cm=confusion_matrix(y_test,y_pred)
        cm=pd.DataFrame(cm,columns=['Predicted No','Predicted Yes'],index=['Actual No','Actual Yes'])
        print(cm)
#         results.ix[model_name,:]=[y_proba]
        skplt.metrics.plot_roc(y_test,y_proba,figsize=(8,6),title='{} ROC Curves'.format(model_name))
        plt.show
        

In [0]:
%pip install scikit-plot

##### No feature engineering
+ without any feature engineering, the model performance is 0.92

In [0]:
na_df=pd.DataFrame(df_185_all1.isna().sum(),columns=['NA_num']).reset_index()
na_df=na_df.sort_values('NA_num',ascending=False)
valid_feat=na_df[na_df['NA_num']<20]['index'].unique()
# below is to make the features visible, you can also just use valid_feat from above as a variable list
valid_feat=['median_flatness', 'count of number of 1s/total windows', 'Favorites', 'pitch_tuning', 'Comments', 'Dislikes', 
            'Likes', 'Views', 'Paper', 'Alphabet', 'Measurements', 'Flyer', 'Electronics', 'Driving License', 
            'Face', 'Plan', 'Handwriting', 'Diagram', 'Document', 'White Board', 'Human', 'Person', 'Word', 'Page', 'Symbol',
            'Number', 'Plot', 'Text', 'wpm', 'Profile faces', 'OtherP',   'Front faces', 'Apostro',
            'Parenth', 'Quote',  'adverb', 'you', 'shehe', 'they', 'ipron', 'article', 'prep', 'auxverb', 'conj',
            'Dash', 'negate', 'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'we', 'i', 'ppron', 'pronoun', 
             'label', 'WC', 'Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'Dic', 'function', 'affect', 'posemo', 'negemo', 'power', 'focuspast', 
            'focuspresent', 'focusfuture', 'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'money', 'informal', 
            'netspeak', 'assent', 'nonflu', 'AllPunc', 'reward', 'achieve', 'social', 'affiliation', 'male', 'cogproc', 
            'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 
            'health', 'drives', 'Number of speakers','Front face','Side profile', 'Voice only']
print(len(valid_feat))
valid_data=df_185_all1[valid_feat].fillna(0)
valid_data.to_csv('../data/training_data_{}feat.csv'.format(len(valid_feat)),index=False)
X=valid_data.drop('label',axis=1)
Y=valid_data['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=88)

In [0]:
evaluate(x_train,x_test,y_train,y_test) 

##### Toss sparse features and kept 73 features

+ kept the 73 features that have less than 100 zeros and got 0.90 ROC Curve

In [0]:
def get_richData(valid_data):
    zeros=pd.DataFrame((valid_data==0).astype(int).sum(axis=0),columns=['num_zeros']).reset_index()
    zeros=zeros.sort_values('num_zeros',ascending=False)
    zeros.to_csv('../data/num_zeros.csv',index=False)
    below_100=zeros[zeros['num_zeros']<100]
    print('{} features that have less than 100 zeros'.format(len(below_100)))
    return lsit(below_100['index'].unique())

In [0]:
below_100=get_richData(valid_data)
# below are the output of the below_100 list, given more features will be added, the variable 'below_100' should supercede the below hard-coded ones
# ['Number of speakers','assent', 'bio', 'leisure', 'Word', 'negemo', 'Plot', 'hear',
#        'negate', 'they', 'nonflu', 'IsFace_digit', 'i', 'work',
#        'informal', 'count of number of 1s/total windows', 'achieve',
#        'reward', 'see', 'you', 'percept', 'motion', 'focuspast',
#        'certain', 'posemo', 'focusfuture', 'power', 'we', 'affect',
#        'discrep', 'insight', 'tentat', 'differ', 'cause',
#        'median_flatness', 'affiliation', 'pitch_tuning', 'Text', 'adj',
#        'compare', 'Views', 'interrog', 'time', 'quant', 'social', 'ppron',
#        'number', 'drives', 'wpm', 'Apostro', 'ipron', 'adverb', 'cogproc',
#        'article', 'Authentic', 'prep', 'auxverb', 'WPS', 'Sixltr', 'Dic',
#        'function', 'Clout', 'Analytic', 'WC', 'verb', 'focuspresent',
#        'pronoun', 'relativ', 'space', 'Tone', 'conj', 'AllPunc', 'label']
FE_nonZero=valid_data[below_100]
X=FE_nonZero.drop('label',axis=1)
Y=FE_nonZero['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=88)
evaluate(x_train,x_test,y_train,y_test)

##### Model with derived features and valid features
+ model performance is 0.98


In [0]:
def text_feat(data,feature):
  data[feature]=data[feature].astype(str)
  data['word_count'+"_"+feature]=data[feature].apply(lambda x: len(x.split(" ")))
  data['char_count'+"_"+feature]=data[feature].str.len()
  def avg_word(word):
   words=word.split( )
   return sum(len(word) for word in words)/len(words)
  data['avg_word'+"_"+feature]=data[feature].apply(lambda x: avg_word(x))
  data['uppercase'+"_"+feature]=data[feature].apply(lambda x: len([x for x in x.split() if x.isupper()]))
  data['lowercase'+"_"+feature]=data[feature].apply(lambda x: len([x for x in x.split() if x.islower()]))
  return data

In [0]:

def get_derived(clean_data_path)
    df_185_all1=pd.read_csv(clean_data_path)
    df_185_all1.columns
    text_feat(df_185_all1,'code_desc')
    text_feat(df_185_all1,'video_desc')
    df_185_all1=text_feat(df_185_all1,'video_title')
    df_185_all1['codeVideoDesc_WC_dif'] =df_185_all1['word_count_code_desc']- df_185_all1['word_count_video_desc']
    df_185_all1['codeVideoDesc_CC_dif']=df_185_all1['char_count_code_desc']- df_185_all1['char_count_video_desc']
    df_185_all1['codeVideoDesc_AvgW_dif']=df_185_all1['avg_word_code_desc']- df_185_all1['avg_word_video_desc']
    df_185_all1['codeVideoDesc_UC_dif']=df_185_all1['uppercase_code_desc']- df_185_all1['uppercase_video_desc']
    df_185_all1['codeVideoDesc_LC_dif']=df_185_all1['lowercase_code_desc']- df_185_all1['lowercase_video_desc']
    df_185_all1['uppercase_video_title']=df_185_all1['video_title'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
    df_185_all1.head(5)
    df_185_all1['lowercase_video_title']=df_185_all1['video_title'].apply(lambda x: len([x for x in x.split() if x.islower()]))
    df_185_all1['codeTitle_WC_dif'] =df_185_all1['word_count_code_desc']- df_185_all1['word_count_video_title']
    df_185_all1['codeTitle_CC_dif']=df_185_all1['char_count_code_desc']- df_185_all1['char_count_video_title']
    df_185_all1['codeTitle_AvgW_dif']=df_185_all1['avg_word_code_desc']- df_185_all1['avg_word_video_title']
    df_185_all1['codeTitle_UC_dif']=df_185_all1['uppercase_code_desc']- df_185_all1['uppercase_video_title']
    df_185_all1['codeTitle_LC_dif']=df_185_all1['lowercase_code_desc']- df_185_all1['lowercase_video_title']
    df_185_all1.to_csv('../data/all_features_wDerived_cleaned.csv',index=False)
    return df_185_all1

In [0]:
clean_data_path='../data/all_features_cleaned.csv'
df_185_all1=get_derived(clean_data_path)
# below is an eyeballed feature list
text_feat=['word_count_code_desc','char_count_code_desc','avg_word_code_desc',
 'uppercase_code_desc','lowercase_code_desc','word_count_video_desc',
 'char_count_video_desc','avg_word_video_desc','uppercase_video_desc',
 'lowercase_video_desc','word_count_video_title','char_count_video_title',
 'avg_word_video_title', 'uppercase_video_title','lowercase_video_title',
 'codeVideoDesc_WC_dif','codeVideoDesc_CC_dif','codeVideoDesc_AvgW_dif',
 'codeVideoDesc_UC_dif','codeVideoDesc_LC_dif','codeTitle_WC_dif',
 'codeTitle_CC_dif','codeTitle_AvgW_dif','codeTitle_UC_dif',
 'codeTitle_LC_dif']
valid_custom_data=df_185_all1[text_feat+valid_feat].fillna(0)
print(valid_custom_data.shape) #(259, 131)
valid_custom_data.to_csv('../data/valid_derived_{}feat.csv'.format(valid_custom_data.shape[1]),index=False)
X=valid_custom_data.drop('label',axis=1)
Y=valid_custom_data['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=88)
evaluate(x_train,x_test,y_train,y_test)

##### Model with text features and features with below 100 zeros
+ model performance is 0.97 (last run on 5/13/2020 after added number of speakers)

In [0]:
valid_custom_NonZero=df_185_all1[text_feat+below_100].fillna(0)
print(valid_custom_NonZero.shape)#(259, 98)
valid_custom_NonZero.to_csv('../data/below100Zero_custom_{}feat.csv'.format(valid_custom_NonZero.shape[1]),index=False)
X=valid_custom_NonZero.drop('label',axis=1)
Y=valid_custom_NonZero['label']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=88)
evaluate(x_train,x_test,y_train,y_test) #0.71